In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import re
import numpy as np
import tensorflow_recommenders as tfrs
import tensorflow as tf

2023-03-12 12:33:11.131426: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 12:33:11.611339: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-12 12:33:11.611393: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-12 12:33:11.611397: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")
model = AutoModel.from_pretrained("sentence-transformers/LaBSE")
data = pd.read_csv("ml-25m/movies.csv")

In [3]:


def remove_pars(x):
    x = str(x)
    return re.sub('[()]', "", x)

titles = [remove_pars(i) for i in data['title']]

def remove_pipes(x):
    x = str(x)
    return re.sub('\|', " ", x)

genres = [remove_pipes(i) for i in data['genres']]

def remove_nulls(a, b, i):
    string_m = a[i] + " " + b[i]
    return re.sub("\(no genres listed\)", "", string_m)

input_string = [remove_nulls(titles, genres, i) for i in range(len(genres))]



In [22]:
from typing import Optional

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

embeddings_file_name = "embeddings/data.csv"
embeddings: Optional[pd.DataFrame] = None

try:
    embeddings = pd.read_csv(embeddings_file_name)
except Exception: 
    embeddings_list = []
    for index, input_ in enumerate(input_string):
        encoded_input = tokenizer(input_, padding=True, truncation=True, max_length=64, return_tensors='pt').to(device)
        with torch.no_grad():
            model_output = model(**encoded_input)
        embeddings = model_output.pooler_output
        embeddings = torch.nn.functional.normalize(embeddings)
        embeddings_list.append(embeddings)
        if index % 1_000  == 0:
            print(str(index))
            
    embeddings_list_tensors = []
    for embedding in embeddings_list:
        tensor_embedding = embedding.cpu()[0].numpy()
        embeddings_list_tensors.append(tensor_embedding)

    embeddings = pd.DataFrame(np.vstack(embeddings_list_tensors))
    embeddings.to_csv("embeddings/data.csv")

if embeddings is None:
    raise RuntimeError("Embeddings are empty")

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000


In [23]:
type(embeddings)

pandas.core.frame.DataFrame

In [24]:
item_tensor = tf.convert_to_tensor(embeddings, dtype=tf.float32)

In [25]:
%%time
scann = tfrs.layers.factorized_top_k.ScaNN(num_leaves=1000, num_leaves_to_search=100, k=round(np.sqrt(len(item_tensor))))
scann.index(item_tensor)

print("All done!")

2023-03-12 12:56:44.919201: I scann/partitioning/partitioner_factory_base.cc:59] Size of sampled dataset for training partition: 62423
2023-03-12 12:56:47.519489: I ./scann/partitioning/kmeans_tree_partitioner_utils.h:88] PartitionerFactory ran in 2.600229416s.


All done!
CPU times: user 1min 31s, sys: 587 ms, total: 1min 31s
Wall time: 12.2 s


In [26]:
test = "Horror films with zombies"
encoded_input = tokenizer(test, padding=True, truncation=True, max_length=64, return_tensors='pt').to(device)
with torch.no_grad():
    model_output = model(**encoded_input)
query = model_output.pooler_output
query = torch.nn.functional.normalize(embeddings)


AttributeError: 'DataFrame' object has no attribute 'norm'

In [ ]:
test_case = scann(np.array(query.cpu()))

In [ ]:


data.iloc[test_case[1].numpy()[0]][0:9]

